In [5]:
import os

import numpy as np
from keras import Sequential, Model
from keras.layers import Dense
from keras.optimizer_v2.adam import Adam

from src.data.loaders.ascad import ASCADData
from src.dlla.hw import encode, dlla_hw
from src.dlla.wegener import make_mlp_wegener, binomial_test
from src.pollution.clock_jitter import clock_jitter
from src.pollution.gaussian_noise import gaussian_noise
from src.pollution.random_delay import random_delay
from src.trace_set.database import Database
from src.trace_set.pollution import Pollution, PollutionType
from src.trace_set.set_hw import TraceSetHW
from src.trace_set.transform import reduce_fixed_fixed, fixed_fixed
from src.trace_set.window import get_windows, extract_traces
from src.tvla.cri import tvla_cri
from src.tvla.tvla import Group

In [17]:
DB = Database.ascad_none
LIMIT_PROF = 200000
LIMIT_ATT = 100000

In [7]:
RAW_DATA = ASCADData.raw()
RAW_TRACES, RAW_LABELS = RAW_DATA['traces'], RAW_DATA['hamming_weights']
Y = RAW_LABELS[:-1][:, 0, 0]

SAMPLE_TRACE = TraceSetHW(DB).profile()[0][0]

In [8]:
WINDOW, WINDOW_CXT = get_windows(RAW_TRACES, SAMPLE_TRACE)

In [9]:
X_CXT = extract_traces(RAW_TRACES, WINDOW_CXT)[:-1]

100%|██████████| 300001/300001 [07:18<00:00, 683.47it/s]


In [10]:
X = X_CXT[:, WINDOW[0]:WINDOW[1]]

In [11]:
PROFILING_MASK = np.ones(len(Y), dtype=bool)
PROFILING_MASK[2::3] = 0

Y_PROF = Y[PROFILING_MASK]
Y_ATT = Y[~PROFILING_MASK]

X_PROF_CXT = X_CXT[PROFILING_MASK]
X_ATT_CXT = X_CXT[~PROFILING_MASK]

X_PROF = X[PROFILING_MASK]
X_ATT = X[~PROFILING_MASK]

X_PROF_CXT.shape, Y_PROF.shape

((200000, 5931), (200000,))

In [ ]:
JITTER_PARAMS = np.arange(0, 21, 1)

def apply_jitter(params):
    for param in params:
        pollution = Pollution(PollutionType.jitter, param)
        out = TraceSetHW(DB, pollution)

        if not os.path.exists(out.path):
            x = clock_jitter(X_PROF_CXT, WINDOW, param)
            x_att = clock_jitter(X_ATT_CXT, WINDOW, param)

            out.create(x, Y_PROF, x_att, Y_ATT)

            x2, y2 = fixed_fixed(x, Y_PROF)
            a, b = x2[~y2], x2[y2]
            order = 2
            _, tvla_p = Group(a, order, True).t_test(Group(b, order, True), order)
            print(f"Clock jitter ({param}): min-p: ({min(tvla_p)}).")

apply_jitter(JITTER_PARAMS)

Clock jitter (2):   0%|          | 0/200000 [00:00<?, ?it/s]

Clock jitter (1): min-p: (2.8472141333494555e-69).


Computing Central Moments: 100%|██████████| 4/4 [00:07<00:00,  1.94s/it]


Clock jitter (2): min-p: (2.509328960319109e-42).


Clock jitter (5):   0%|          | 38/200000 [00:00<08:46, 379.44it/s]

...

Clock jitter (6):   0%|          | 46/200000 [00:00<07:18, 455.84it/s]

Clock jitter (5): min-p: (8.97684110831795e-20).


Clock jitter (7):   0%|          | 50/200000 [00:00<06:46, 492.34it/s]

Clock jitter (6): min-p: (5.927403449010637e-19).


Clock jitter (7):  32%|███▏      | 63519/200000 [02:08<04:29, 505.82it/s]

In [13]:
DELAY_PARAMS = np.arange(0, .80, .05)
A = 5
B = 3
DELAY_AMP = 10

def apply_delay(params):
    for param in params:
        pollution = Pollution(PollutionType.delay, param)
        out = TraceSetHW(DB, pollution)

        if not os.path.exists(out.path):
            x = random_delay(X_PROF, A, B, DELAY_AMP, param)
            x_att = random_delay(X_ATT, A, B, DELAY_AMP, param)

            out.create(x, Y_PROF, x_att, Y_ATT)

            x2, y2 = fixed_fixed(x, Y_PROF)
            a, b = x2[~y2], x2[y2]
            order = 2
            _, tvla_p = Group(a, order, True).t_test(Group(b, order, True), order)
            print(f"Random delay ({param}): min-p: ({min(tvla_p)}).")

# apply_delay(DELAY_PARAMS)

In [14]:
GAUSS_PARAMS = np.arange(0, 32, 2)

### TVLA vs. DL-LA

In [15]:
ORDER = 2

def store_results(database: Database, method: str, pollution: Pollution, p):
    file_name = f"results_{database.name}.csv"
    with open(file_name, 'a') as f:
        f.write(f"{method};{pollution.type.name};{pollution.parameter};{p}\n")

In [18]:
def prepare_traces_dl(x, y, x_att, y_att):
    """
    Normalizes the traces, one-hot encodes the labels.
    Returns profiling traces, labels and attack traces, labels.
    """
    prof_mean, prof_std = x.mean(axis=0), x.std(axis=0)
    norm_x = (x - prof_mean) / prof_std
    norm_x_att = (x_att - prof_mean) / prof_std

    return norm_x, encode(y), norm_x_att, encode(y_att)


def build_mlp(x, y, params):
    mdl = Sequential()
    mdl.add(Dense(100, activation=params['activation'], input_shape=(x.shape[1],)))
    mdl.add(Dense(100, activation=params['activation']))
    mdl.add(Dense(100, activation=params['activation']))
    mdl.add(Dense(100, activation=params['activation']))
    mdl.add(Dense(9, activation='softmax'))

    mdl.compile(optimizer=params['optimizer'], loss=params['losses'], metrics=['accuracy'])

    out = mdl.fit(x, y, shuffle=True, batch_size=params['batch_size'], epochs=params['epochs'], verbose=False)

    return out, mdl


def make_mlp(x, y):
    return build_mlp(x, y, {
        'activation': 'relu',
        'optimizer': Adam(learning_rate=0.001),
        'losses': 'categorical_crossentropy',
        'batch_size': 150,
        'epochs': 5
    })[1]

def wegener_p(mdl: Model, x_att: np.ndarray, y_att: np.ndarray):
    predictions = mdl.predict(x_att).argmax(axis=1)
    labels = y_att.argmax(axis=1)

    correct = np.sum(predictions == labels)
    total = len(predictions)
    # print(total, correct)

    return binomial_test(total, correct)

def la_benchmark(db: Database, pollution_type: PollutionType, params):
    for param in params:
        pollution = Pollution(pollution_type, param)
        print("Load traces         ", end="\r")
        trace_set = TraceSetHW(db, pollution, (LIMIT_PROF, LIMIT_ATT))

        if os.path.exists(trace_set.path):
            print("Prepare traces (1/2)", end="\r")
            x9, y9, x9_att, y9_att = prepare_traces_dl(*trace_set.profile(), *trace_set.attack())
            print("Prepare traces (2/2)", end="\r")
            (x2, y2), (x2_att, y2_att) = reduce_fixed_fixed(x9, y9), reduce_fixed_fixed(x9_att, y9_att)

            print("Make model (1/2)    ", end="\r")
            mdl9 = make_mlp(x9, y9)
            print("Make model (2/2)    ", end="\r")
            mdl2 = make_mlp_wegener(x2, y2, False)

            print("TVLA                ", end="\r")
            tvla_ps = np.min(tvla_cri(trace_set, ORDER), axis=1)

            print("Predict             ", end="\r")
            dlla9_p, _ = dlla_hw(mdl9, x9_att, y9_att)
            dlla2_p = wegener_p(mdl2, x2_att, y2_att)

            print(f"{pollution_type} ({param}). TVLA ({tvla_ps}). DLLA9 ({dlla9_p}). DLLA2 ({dlla2_p})", end="\r")

            for order, p in enumerate(tvla_ps):
                if order > 0:
                    store_results(db, f"cri_tvla_{order}", pollution, p)

            store_results(db, "dlla9", pollution, dlla9_p)
            store_results(db, "dlla2", pollution, dlla2_p)

        print()

while True:
    la_benchmark(DB, PollutionType.jitter, JITTER_PARAMS)
    # la_benchmark(DB, PollutionType.delay, DELAY_PARAMS)
    # la_benchmark(DB, PollutionType.gauss, GAUSS_PARAMS)


PollutionType.jitter (0). TVLA ([1.00000000e+000 0.00000000e+000 2.57266244e-213]). DLLA9 (0.0). DLLA2 (0.0)
PollutionType.jitter (4). TVLA ([1.00000000e+00 2.61143203e-15 5.58863915e-29]). DLLA9 (4.1138213087011504e-27). DLLA2 (7.361934398680176e-21)
PollutionType.jitter (8). TVLA ([1.00000000e+00 2.76423834e-06 1.57707532e-12]). DLLA9 (0.0007867806938174644). DLLA2 (0.052087270223763485)
PollutionType.jitter (12). TVLA ([1.00000000e+00 3.87689506e-09 1.72874245e-13]). DLLA9 (4.981118371603464e-07). DLLA2 (0.006951163230122877)
PollutionType.jitter (16). TVLA ([1.00000000e+00 5.66778564e-05 3.42683380e-11]). DLLA9 (0.24049161860070686). DLLA2 (0.026769210762774776)
PollutionType.jitter (20). TVLA ([1.00000000e+00 6.08621014e-06 7.58471003e-12]). DLLA9 (0.00046344532673803186). DLLA2 (0.014210058421155502)
PollutionType.jitter (24). TVLA ([1.00000000e+00 2.69324280e-07 1.96468399e-10]). DLLA9 (0.12815840058806877). DLLA2 (0.027233834610860177)
PollutionType.jitter (28). TVLA ([1.000000

KeyboardInterrupt: 